In [1]:
# Import required libs
import os
from utilities import jpg_to_mp4
from dataset import VideoDataset

In [2]:
# Set paths
HOME = os.getcwd()
data_path = os.path.join(HOME,"Data")

In [8]:
# Uncomment the line below to convert all .jpg sequences in  into .mp4 videos @ 24 fps
#jpg_to_mp4(data_path, frame_rate=24)

In [3]:
# Create video dataset
dataset = VideoDataset(data_path)

for video in dataset:
    print(video)

['c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_1\\seq_1_view_1.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_1\\seq_1_view_2.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_1\\seq_1_view_3.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_1\\seq_1_view_4.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_1\\seq_1_view_5.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_1\\seq_1_view_6.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_1\\seq_1_view_7.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_1\\seq_1_view_8.mp4']
['c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_10\\seq_10_view_1.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_10\\seq_10_view_2.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_10\\seq_10_view_3.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_10\\seq_10_view_4.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_10\\seq_10_view_5.mp4', 'c:\\Users\\umerh\\Desktop\\Project\\Data\\seq_10\\seq_10_view_6.mp4', 'c:\\Users\\umerh\\D

In [4]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

# temporary fix for some weird locale bug
import locale
locale.getpreferredencoding = lambda: "UTF-8"

HOME: c:\Users\umerh\Desktop\Project


In [5]:
# either one of these should work
%cd {HOME}
try:
  import groundingdino
  from groundingdino.util.inference import Model as GroundingDINOModel
except ImportError:
  import GroundingDINO
  from GroundingDINO.groundingdino.util.inference import Model as GroundingDINOModel

c:\Users\umerh\Desktop\Project


In [8]:
%cd {HOME}/Tracking-Anything-with-DEVA

import os
from os import path
from argparse import ArgumentParser

import torch
import numpy as np

from deva.model.network import DEVA
from deva.inference.inference_core import DEVAInferenceCore
from deva.inference.result_utils import ResultSaver
from deva.inference.eval_args import add_common_eval_args, get_model_and_config
from deva.inference.demo_utils import flush_buffer
from deva.ext.ext_eval_args import add_ext_eval_args, add_text_default_args
from deva.ext.grounding_dino import get_grounding_dino_model
from deva.ext.with_text_processor import process_frame_with_text as process_frame

from tqdm import tqdm
import json

torch.autograd.set_grad_enabled(False)

# for id2rgb
np.random.seed(42)

# default parameters
parser = ArgumentParser()
add_common_eval_args(parser)
add_ext_eval_args(parser)
add_text_default_args(parser)

# load model and config
args = parser.parse_args([])
cfg = vars(args)
cfg['enable_long_term'] = True

# Load our checkpoint
deva_model = DEVA(cfg).cuda().eval()
if args.model is not None:
    model_weights = torch.load(args.model)
    deva_model.load_weights(model_weights)
else:
    print('No model loaded.')

gd_model, sam_model = get_grounding_dino_model(cfg, 'cuda')

c:\Users\umerh\Desktop\Project\Tracking-Anything-with-DEVA


"deva_model = DEVA(cfg).cuda().eval()\nif args.model is not None:\n    model_weights = torch.load(args.model)\n    deva_model.load_weights(model_weights)\nelse:\n    print('No model loaded.')\n\ngd_model, sam_model = get_grounding_dino_model(cfg, 'cuda')"

In [9]:
cfg['enable_long_term_count_usage'] = True
cfg['max_num_objects'] = 50
cfg['size'] = 480
cfg['DINO_THRESHOLD'] = 0.35
cfg['amp'] = True
cfg['chunk_size'] = 4
cfg['detection_every'] = 5
cfg['max_missed_detection_count'] = 10
cfg['sam_variant'] = 'original'
cfg['temporal_setting'] = 'semionline' # semionline usually works better than online
cfg['pluralize'] = True

In [11]:
from utilities import get_segmented_output_path

SOURCE_VIDEO_PATH = dataset[0][0]
CLASSES = ['black sticks', 'hands', 'plastic','person','rectangular black object'] ## detecting hands has higher acc with person, small plastic is hard to detect

# Path to save output video
OUTPUT_VIDEO_PATH = get_segmented_output_path(SOURCE_VIDEO_PATH)

In [12]:
%cd {HOME}/Tracking-Anything-with-DEVA

from deva.ext.with_text_processor import process_frame_with_text as process_frame_text
import tempfile
import cv2

cfg['prompt'] = '.'.join(CLASSES)

deva = DEVAInferenceCore(deva_model, config=cfg)
deva.next_voting_frame = cfg['num_voting_frames'] - 1
deva.enabled_long_id()

# obtain temporary directory
result_saver = ResultSaver(None, None, dataset='gradio', object_manager=deva.object_manager)
writer_initizied = False

cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
ti = 0
# only an estimate
with torch.cuda.amp.autocast(enabled=cfg['amp']):
    with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))) as pbar:
        while (cap.isOpened()):
            ret, frame = cap.read()
            if ret == True:
                if not writer_initizied:
                    h, w = frame.shape[:2]
                    writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'vp80'), fps, (w, h))
                    writer_initizied = True
                    result_saver.writer = writer

                process_frame_text(deva,
                                    gd_model,
                                    sam_model,
                                    'null.png',
                                    result_saver,
                                    ti,
                                    image_np=frame)
                ti += 1
                if ti == 50:
                    break
                pbar.update(1)
            else:
                break
    flush_buffer(deva, result_saver)
writer.release()
cap.release()
deva.clear_buffer()

c:\Users\umerh\Desktop\Project\Tracking-Anything-with-DEVA


  0%|          | 2/1584 [00:03<40:13,  1.53s/it]  

Restricted license - for non-production use only - expires 2025-11-24


  3%|▎         | 49/1584 [00:19<10:13,  2.50it/s]
